In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from python_graphql_client import GraphqlClient

In [4]:
# Instantiate the client with an endpoint.
client = GraphqlClient(endpoint="https://api.thegraph.com/subgraphs/name/nounsdao/nouns-subgraph")

In [117]:
# Create the query string and variables required for the request.
query = """
    {
  nouns(first: 1000, orderBy: id) {
    id,
    owner,
    votes {
        proposal {
            id,
            proposer{id},
            proposalThreshold,
            quorumVotes,
            forVotes,
            againstVotes,
            abstainVotes,
            description,
            status,
            votes{id,support,votes,reason}

        },
        supportDetailed
    }
  }
}
"""
# Synchronous request
data = client.execute(query=query)

# For noun in data["data"]["nouns"] get each ['votes'] ['proposal'] and add them all to a list
proposals = []
for noun in data["data"]["nouns"]:
    for vote in noun["votes"]:
        proposals.append(vote["proposal"])


In [104]:
proposals_df = pd.DataFrame(proposals)
# each row of ['proposer'] is a dictionary {'id':'...'}, so we need to just keep the 'id'
proposals_df["proposer"] = proposals_df["proposer"].apply(lambda x: x["id"])


In [105]:
# change default row and column display limit to none
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [111]:
proposals_df = proposals_df.loc[proposals_df.astype(str).drop_duplicates().index]
proposals_df.shape


(93, 10)

In [147]:
proposals_df.head()

,id,proposer,proposalThreshold,quorumVotes,forVotes,againstVotes,abstainVotes,description,status,votes,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32
0,1,0x2573c60a6d127755aa2dc85e342f7da2378a0cc5,0,1,7,0,0,# Donate 5 ETH each to 6 Charities\n\n*Past Di...,EXECUTED,[{'id': '0x008c84421da5527f462886cec43d2717b68...,{'id': '0x008c84421da5527f462886cec43d2717b686...,{'id': '0x2536c09e5f5691498805884fa37811be3b2b...,{'id': '0x2573c60a6d127755aa2dc85e342f7da2378a...,{'id': '0x54be3a794282c030b15e43ae2bb182e14c40...,{'id': '0xae7f458667f1b30746354abc3157907d9f6f...,{'id': '0xf476cd75be8fdd197ae0b466a2ec2ae44da4...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,10,0xf476cd75be8fdd197ae0b466a2ec2ae44da41897,0,6,21,0,0,# Commission Jonathan Mann (@songadayman) to m...,EXECUTED,[{'id': '0x008c84421da5527f462886cec43d2717b68...,{'id': '0x008c84421da5527f462886cec43d2717b686...,{'id': '0x2573c60a6d127755aa2dc85e342f7da2378a...,{'id': '0x2e5ccffbae934a9cb7e061fca7af01946e4e...,{'id': '0x43a330dec81bbd5e21f41c6b8354e54d481e...,{'id': '0xae7f458667f1b30746354abc3157907d9f6f...,{'id': '0xf476cd75be8fdd197ae0b466a2ec2ae44da4...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,11,0x6f9e3976fa3b5b22761fe5d635e1f0d9d9aeb85d,0,6,20,0,0,# Nouns Comic\n\n# **Nouns Comic** \n\n**Nouns...,EXECUTED,[{'id': '0x008c84421da5527f462886cec43d2717b68...,{'id': '0x008c84421da5527f462886cec43d2717b686...,{'id': '0x17beaa158e68ff9225b4c85151831c7a0d8f...,{'id': '0x2573c60a6d127755aa2dc85e342f7da2378a...,{'id': '0x33daf8e90cd1f214cc25a1865649e847254f...,{'id': '0x6c0cf880cb20eefabfb09341fba9e2bd29ad...,{'id': '0x6f9e3976fa3b5b22761fe5d635e1f0d9d9ae...,{'id': '0x75ee6eb3d8dacf41ee2e5307090b197d3e1c...,{'id': '0x9646e0f7544b8ee3649e7ac77eef0cf5b53b...,{'id': '0x9cb8c1463471ee4d8dd6887035f4835a1745...,{'id': '0xa1e4f7dc1983fefe37e2175524ebad87f1c7...,{'id': '0xc3b80922832450aabafe6aae8a6aaf1e5e61...,{'id': '0xd049b3064990869c9f73bd7896271d83325d...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,12,0xae7f458667f1b30746354abc3157907d9f6fd15e,0,7,20,0,0,**Perpetual Glasses to Kids In Need V2**\n===\...,EXECUTED,[{'id': '0x2573c60a6d127755aa2dc85e342f7da2378...,{'id': '0x2573c60a6d127755aa2dc85e342f7da2378a...,{'id': '0x2e5ccffbae934a9cb7e061fca7af01946e4e...,{'id': '0x33daf8e90cd1f214cc25a1865649e847254f...,{'id': '0x48e3e06520ff208746b8555076f7af3e71af...,{'id': '0x6c0cf880cb20eefabfb09341fba9e2bd29ad...,{'id': '0x6f9e3976fa3b5b22761fe5d635e1f0d9d9ae...,{'id': '0x75ee6eb3d8dacf41ee2e5307090b197d3e1c...,{'id': '0xa387c87102fc0b51939d0cf9a86f7edd0529...,{'id': '0xae7f458667f1b30746354abc3157907d9f6f...,{'id': '0xf476cd75be8fdd197ae0b466a2ec2ae44da4...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,14,0xf476cd75be8fdd197ae0b466a2ec2ae44da41897,0,8,25,0,0,# Brave Sponsored Takeover during NFT NYC\n\n#...,EXECUTED,[{'id': '0x008c84421da5527f462886cec43d2717b68...,{'id': '0x008c84421da5527f462886cec43d2717b686...,{'id': '0x2573c60a6d127755aa2dc85e342f7da2378a...,{'id': '0x3612b2e93b49f6c797066ca8c38b7f522b32...,{'id': '0x48e3e06520ff208746b8555076f7af3e71af...,{'id': '0x696ed7b26f4b019ceec78dc8b9140ad64a6f...,{'id': '0x6f9e3976fa3b5b22761fe5d635e1f0d9d9ae...,{'id': '0xa387c87102fc0b51939d0cf9a86f7edd0529...,{'id': '0xa5960ad956acbb8addebdc29e5bee014f201...,{'id': '0xca6caedcc79d4d6541b3f82dc7754a14d9b1...,{'id': '0xd049b3064990869c9f73bd7896271d83325d...,{'id': '0xea9e9ae3bd4a7f771eec5193c0498f90fe2d...,{'id': '0xf3d5e8f1cb8cc717661d6a8e692b60cdd1b4...,{'id': '0xf476cd75be8fdd197ae0b466a2ec2ae44da4...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [161]:
# Create a new dataframe then loop through all rows and for each row loop through all 'votes'
# Each vote that has a 'reason' we will append a new row to the new dataframe
# The new dataframe will have the columns 'proposal_id','quorumVotes','forVotes','againstVotes', 'abstainVotes','description','status','voter id','voter support','votes','reason'

reasons_df = pd.DataFrame()
for index, row in proposals_df.iterrows():
    for vote in row["votes"]:
        if vote["reason"] is not None:
            if vote['support']:
                # append using concat to add a new row to the dataframe into the correct columns saying "I support this proposal"
                reasons_df = pd.concat([reasons_df, pd.DataFrame([[row["id"], row["quorumVotes"], row["forVotes"], row["againstVotes"], row["abstainVotes"],"# Proposal: " + row["description"], row["status"], vote["id"], vote["support"], vote["votes"],"## I support this proposal: " + vote["reason"]]])], ignore_index=True)
            else:
                # append using concat to add a new row to the dataframe into the correct columns saying "I do not support this proposal"
                reasons_df = pd.concat([reasons_df, pd.DataFrame([[row["id"], row["quorumVotes"], row["forVotes"], row["againstVotes"], row["abstainVotes"],"# Proposal: " + row["description"], row["status"], vote["id"], vote["support"], vote["votes"],"## I do not support this proposal: " + vote["reason"]]])], ignore_index=True)

# rename the columns
reasons_df.columns = ['proposal_id','quorumVotes','forVotes','againstVotes', 'abstainVotes','description','status','voter id','voter support','votes','reason']

In [162]:
# print the shape of the new dataframe
print(reasons_df.shape)
# print all the unique proposal ids sorted low to high
print(reasons_df["proposal_id"].sort_values().unique().tolist())
# print how many unique proposal ids there are
print(len(reasons_df["proposal_id"].sort_values().unique().tolist()))

(271, 11)
['44', '47', '49', '53', '54', '55', '56', '57', '58', '59', '60', '61', '62', '63', '64', '65', '67', '68', '69', '70', '71', '72', '73', '74', '75', '77', '78', '81', '82', '83', '84', '85', '86', '87', '89', '90', '91', '92', '93', '94', '95', '97', '98', '99']
44


In [163]:
reasons_df.head()

,proposal_id,quorumVotes,forVotes,againstVotes,abstainVotes,description,status,voter id,voter support,votes,reason
0,44,24,78,1,1,# Proposal: # Nouns Acquisition Committee\n\n#...,EXECUTED,0xcb6fbf9b4e822bed1a8365c0c32e66bd7a779bae-44,True,0,## I support this proposal: I am in favor
1,49,26,33,9,28,# Proposal: # Additional Funding for Nouns Acq...,EXECUTED,0x5306c064f74b2c45d3f1afae90cf0d74f7523fe4-49,False,0,## I do not support this proposal: stated purp...
2,53,26,75,0,1,# Proposal: # Contribute 6.9E to funding Calla...,EXECUTED,0xcc2688350d29623e2a0844cc8885f9050f0f6ed5-53,True,7,## I support this proposal: we liked it
3,57,27,48,8,16,# Proposal: # Bring luxury Noun sunglasses to ...,EXECUTED,0x008c84421da5527f462886cec43d2717b686a7e4-57,False,5,## I do not support this proposal: For the Pro...
4,57,27,48,8,16,# Proposal: # Bring luxury Noun sunglasses to ...,EXECUTED,0x2573c60a6d127755aa2dc85e342f7da2378a0cc5-57,True,26,## I support this proposal: ⌐◨-◨


In [164]:
# print the full "reason" of any row with a duplicate reason
reasons_df.loc[reasons_df.duplicated(subset="reason", keep=False)]

,proposal_id,quorumVotes,forVotes,againstVotes,abstainVotes,description,status,voter id,voter support,votes,reason
4,57,27,48,8,16,# Proposal: # Bring luxury Noun sunglasses to ...,EXECUTED,0x2573c60a6d127755aa2dc85e342f7da2378a0cc5-57,True,26,## I support this proposal: ⌐◨-◨
36,62,27,71,0,0,# Proposal: # Prop House Continuation\n\n**tl;...,EXECUTED,0x2573c60a6d127755aa2dc85e342f7da2378a0cc5-62,True,25,## I support this proposal: ⌐◨-◨
184,92,33,126,1,0,# Proposal: # David Horvath / Nouns Studio1 ex...,EXECUTED,0xcc2688350d29623e2a0844cc8885f9050f0f6ed5-92,True,13,## I support this proposal: ⌐◨-◨
229,98,34,42,0,6,# Proposal: # NOUNS IN RIO - Nouns to be a par...,QUEUED,0xcc2688350d29623e2a0844cc8885f9050f0f6ed5-98,True,12,## I support this proposal: ⌐◨-◨
236,99,35,53,2,0,"# Proposal: # Sailing PR campaign, Korea Block...",ACTIVE,0xa68fdb0c7ecdc404637c5c06bab82a7bf00609f3-99,True,1,## I support this proposal: ⌐◨-◨
245,56,27,39,1,2,# Proposal: # FUN Noggles Stage 2\n\n### Shark...,EXECUTED,0x8323f1c687f7e2296ec71ee3549a7430ea7ec730-56,True,1,## I support this proposal: ⌐◨-◨


In [165]:
# rename 'description' to 'prompt' and 'reason' to 'completion'
reasons_df.rename(columns={"description": "prompt", "reason": "completion"}, inplace=True)

In [166]:
# save the dataframe to a csv file
reasons_df.to_csv("nouns_vote_reasons.csv", index=False)